In [2]:
%matplotlib inline

import datacube
import rasterio.crs
import geopandas as gpd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from datacube.utils import geometry
from dea_tools.spatial import xr_rasterize
from dea_tools.plotting import rgb

In [3]:
dc = datacube.Datacube(app='rabbit_colours')

In [6]:
gdf = gpd.read_file('rabbits_500m.shp')

# Create a csv file to save results 
with open('rabbit_colours.csv', 'w') as f:
    f.write('Id,Year,Red,Green,Blue\n')

# Loop through polygons
for index, row in gdf.iterrows():
    
    # Get site ID
    ID = str(row['Id'])
    
    # Extract the feature's geometry as a datacube geometry object
    geom = geometry.Geometry(geom=row.geometry, crs=gdf.crs)
    
    # Loop over years and load geomedian landsat data
    for y in range(2013, 2023):
        ls8_ds = dc.load(product='ga_ls8c_nbart_gm_cyear_3',
                         measurements=['red', 'green', 'blue'],
                         time=(str(y), str(y)),
                         output_crs='EPSG:3577',
                         geopolygon=geom)

        # Generate a polygon mask to keep only data within the polygon
        mask = xr_rasterize(gdf.iloc[[index]], ls8_ds)
    
        # Mask dataset to set pixels outside the polygon to `NaN`
        l8_site = ls8_ds.where(mask)
    
        # Obtain mean and std dev of pixels
        stats = []
        for b in ['red', 'green', 'blue']:
            d = l8_site[b].values
            d = d[(d != -1) & (np.isnan(d) != 1)]
            if d.size > 1:
                stats.append('%.2f'%np.mean(d))
            else:
                stats.append('No valid pixels')
        
        # Write to csv
        with open('rabbit_colours.csv', 'a') as f:
            f.write('%s,%s,%s\n'%(ID, y, ','.join(stats)))
